# E‑Commerce Business Data Cleaning & Validation
This notebook prepares dirty, unequal, executive‑grade e‑commerce data for analysis by validating keys, fixing integrity issues, handling nulls & duplicates, standardizing values, and removing outliers using the IQR method.

## Objectives

- Ensure Primary Key (PK) uniqueness & validity

- Validate Foreign Key (FK) relationships across tables

- Handle nulls, duplicates, invalid values

- Standardize inconsistent categorical data

- Remove outliers using IQR (analysis‑safe)

- Deliver clean, analysis‑ready datasets

In [23]:
# Import libraries

import pandas as pd
import numpy as np

In [24]:
# Load files

sales = pd.read_csv("data/sales_transactions.csv")
customers = pd.read_csv("data/customers.csv")
products = pd.read_csv("data/products.csv")
stores = pd.read_csv("data/stores.csv")

In [30]:
# Taking the preview 
"""
table = {
    "Sales": sales,
    "Customers": customers,
    "Products": products,
    "Stores": stores
}
for name, df in table.items():
    print(f"\n{name} Shape:", df.shape)
    display(df.head())"""

customers['State'].unique()

array(['KY', 'FL', 'OR', 'AL', 'PA', 'DC', 'CA', 'VI', 'NJ', 'NV', 'SD',
       'NE', 'MI', 'WY', 'CT', 'GA', 'MN', 'IA', 'ND', 'CO', 'WI', 'LA',
       'DE', 'HI', 'RI', 'NC', 'PR', 'ID', 'NH', 'MH', 'NM', 'NY', 'WA',
       'AZ', 'AK', 'WV', 'AS', 'MO', 'VT', 'TX', 'GU', 'OH', 'MT', 'IN',
       'FM', 'MP', 'IL', 'KS', 'PW', 'MS', 'SC', 'AR', 'MA', 'TN', 'ME',
       'OK', 'VA', 'UT', 'MD'], dtype=object)

### Primary Key Uniqueness Check

In [11]:
def pk_check(df, pk):

    return {
    "Total Rows": len(df),
    "Unique PKs": df[pk].nunique(),
    "Duplicate PKs": df.duplicated(pk).sum(),
    "Null PKs": df[pk].isna().sum()
}


pk_summary = {
"Sales": pk_check(sales, "Order_ID"),
"Customers": pk_check(customers, "Customer_ID"),
"Products": pk_check(products, "Product_ID"),
"Stores": pk_check(stores, "Store_ID")
}


pd.DataFrame(pk_summary).T

,Total Rows,Unique PKs,Duplicate PKs,Null PKs
Sales,18150,18000,150,0
Customers,825,800,25,0
Products,120,120,0,0
Stores,25,25,0,0


In [12]:
sales = sales.drop_duplicates(subset="Order_ID")
customers = customers.drop_duplicates(subset="Customer_ID")
products = products.drop_duplicates(subset="Product_ID")
stores = stores.drop_duplicates(subset="Store_ID")

### Foreign Key Validation Check


In [13]:
sales = sales[sales["Customer_ID"].isin(customers["Customer_ID"])]
sales = sales[sales["Product_ID"].isin(products["Product_ID"])]
sales = sales[sales["Store_ID"].isin(stores["Store_ID"])]

### Null Value Handling

In [14]:
sales.isna().sum()

Order_ID           0
Order_Date        49
Customer_ID        0
Product_ID         0
Store_ID           0
Quantity           0
Unit_Price         0
Discount        5558
Revenue            0
Cost               0
Profit             0
Order_Status       0
dtype: int64

In [15]:
sales = sales.dropna(subset=["Order_Date", "Customer_ID"])
sales["Discount"] = sales["Discount"].fillna(0)

### Data Standardization

In [16]:
products["Category"] = products["Category"].str.lower()
products["Category"] = products["Category"].replace({
    "electronics": "electronics",
    "elec": "electronics",
    "accessories": "accessories"
})

In [17]:
state_map = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas",
    "CA": "California", "CO": "Colorado", "CT": "Connecticut", "DE": "Delaware",
    "FL": "Florida", "GA": "Georgia", "HI": "Hawaii", "ID": "Idaho",
    "IL": "Illinois", "IN": "Indiana", "IA": "Iowa", "KS": "Kansas",
    "KY": "Kentucky", "LA": "Louisiana", "ME": "Maine", "MD": "Maryland",
    "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi",
    "MO": "Missouri", "MT": "Montana", "NE": "Nebraska", "NV": "Nevada",
    "NH": "New Hampshire", "NJ": "New Jersey", "NM": "New Mexico", "NY": "New York",
    "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio", "OK": "Oklahoma",
    "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina",
    "SD": "South Dakota", "TN": "Tennessee", "TX": "Texas", "UT": "Utah",
    "VT": "Vermont", "VA": "Virginia", "WA": "Washington", "WV": "West Virginia",
    "WI": "Wisconsin", "WY": "Wyoming"
}

customers["State"] = customers["State"].map(state_map)
stores["State"] = stores["State"].map(state_map)

C:\Users\imanc\AppData\Local\Temp\ipykernel_22076\2319967723.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers["State"] = customers["State"].map(state_map)


In [18]:
# Profit is not standardize as it can be Negative

numeric_cols = ["Quantity", "Unit_Price", "Revenue", "Cost"]


for col in numeric_cols:
    sales = sales[sales[col] >= 0]

### Outlier Detection and Removel (IQR)

In [19]:
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[column] >= lower) & (df[column] <= upper)]

In [20]:
for col in ["Quantity", "Unit_Price", "Revenue", "Profit"]:
    sales = remove_outliers_iqr(sales, col)

In [21]:
sales["Order_Date"] = pd.to_datetime(sales["Order_Date"], errors="coerce")
sales = sales.dropna(subset=["Order_Date"])

In [22]:
sales.to_csv("clean/sales.csv", index=False)
customers.to_csv("clean/customers.csv", index=False)
products.to_csv("clean/products.csv", index=False)
stores.to_csv("clean/stores.csv", index=False)

### Outcome

- Referential integrity enforced

- Duplicates & nulls removed

- Categories standardized

- Outliers handled using IQR

- Analysis‑ready datasets produced